#### Student Name: Jiayi Zhang
#### Student ID: A14533542
#### Date: July.18th 2020

# CSE 190 Final Project

### Project Title: Generate K-Pop Music Base On RNN Model and Music Structure 



The dataset is quite small (around 90 files) and some of them includes the entire song that played piano on different tracks and some of them includes only an fragment of the song (mostly Intro or Chorus of the song) that's also mainly played by piano. The intention of the project is to separate the dataset into different segments based on the most simplified music structure that's commonly used in K-Pop music. (Riff/Intro->Verse1->Pre-Chorus->Chorus->Verse2->Pre-Chorus->Chorus->Riff/Outro) Verse1 and Verse2 will be both considered as Verse out of simplicity. 


In [2]:
import numpy as np
import pandas as pd
from collections import Counter
from numpy.random import multinomial as randm
from numpy import where
import scipy.signal as si
import IPython.display as ipd
import librosa
import scipy
from matplotlib import patches
import librosa.display as ld
import music21
from music21 import midi as midi21
import mido
from jchord.progressions import ChordProgression
import copy
import matplotlib.pyplot as plt
from scipy.io import wavfile
from numpy.linalg import svd
from scipy.stats.mstats import gmean
from matplotlib import rcParams
import os
import glob
import sys
import pickle
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Lambda
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [72]:
#I'm using the network straight from Assignment 3 because it will be hard to create my own network. 
def create_network(network_input, n_vocab):

    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(
        512,
        recurrent_dropout=0.3,
    ))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Lambda(lambda network_input: network_input / 0.6))
    model.add(Activation('softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy')

    return model

In [234]:
def create_network(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    print(n_vocab)
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Lambda(lambda network_input: network_input / 0.6))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    print(model.summary())
    
    return model

In [187]:
def put_note(total, stamp, element, Intro, Verse, Chorus, Outro):
    if(stamp <= total*0.05):
        Intro.append(str(element.pitch))
    elif(stamp <= total*0.22):
        Verse.append(str(element.pitch))
    elif(stamp <= total*0.39):
        Chorus.append(str(element.pitch))
    elif(stamp <= total*0.56):
        Verse.append(str(element.pitch))
    elif(stamp <= total*0.73):
        Chorus.append(str(element.pitch))
    elif(stamp <= total*0.9):
        Verse.append(str(element.pitch))
    else:
        Outro.append(str(element.pitch))

In [188]:
def put_chord(total, stamp, element, Intro, Verse, Chorus, Outro):
    if(stamp <= total*0.05):
        Intro.append('.'.join(str(n) for n in [p.midi for p in element.pitches]))
    elif(stamp <= total*0.22):
        Verse.append('.'.join(str(n) for n in [p.midi for p in element.pitches]))
    elif(stamp <= total*0.39):
        Chorus.append('.'.join(str(n) for n in [p.midi for p in element.pitches]))
    elif(stamp <= total*0.56):
        Verse.append('.'.join(str(n) for n in [p.midi for p in element.pitches]))
    elif(stamp <= total*0.73):
        Chorus.append('.'.join(str(n) for n in [p.midi for p in element.pitches]))
    elif(stamp <= total*0.9):
        Verse.append('.'.join(str(n) for n in [p.midi for p in element.pitches]))
    else:
        Outro.append('.'.join(str(n) for n in [p.midi for p in element.pitches]))

In [189]:
def group_by_timestamp():
    
    Intro = []
    Verse = []
    Chorus = []
    Outro = []
    
    for file in glob.glob("COPY/*.mid"):
        midi = converter.parse(file)
        print("Parsing %s" % file)
        notes_to_parse = None
        try:
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts['Piano'].recurse()
        except:
            notes_to_parse = midi.flat.notes
        
        totalLength = midi.highestTime
        
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                put_note(totalLength, element.offset, element, Intro, Verse, Chorus, Outro)
            elif isinstance(element, chord.Chord):
                put_chord(totalLength, element.offset, element, Intro, Verse, Chorus, Outro)
                
    pickle.dump(Intro, open('Intro.p', 'wb'))
    pickle.dump(Verse, open('Verse.p', 'wb'))
    pickle.dump(Chorus, open('Chorus.p', 'wb'))
    pickle.dump(Outro, open('Outro.p', 'wb'))
    
    return(Intro, Verse, Chorus, Outro)

In [167]:
def get_offset_difference():
    
    offset = []
    difference = []
    
    for file in glob.glob("COPY/*.mid"):
        midi = converter.parse(file)
        print("Parsing %s" % file)
        offset_to_parse = None
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            offset_to_parse = s2.parts['Piano'].recurse() 
        except: # file has notes in a flat structure
            offset_to_parse = midi.flat.notes
            
        #Store each notes and the duration it's played in the music as a key-value pair
        for element in offset_to_parse:
            offset.append(element.offset)
        
        for index in range(len(offset)-1):
            difference.append(offset[index+1]-offset[index])
            
            
    pickle.dump(difference, open('offset.p', 'wb'))
    
    return difference

In [190]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 40

    pitchnames = sorted(set(item for item in notes))

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [235]:
def train_duration_network():
    """ Train a Neural Network to generate music """
    durations = get_offset_difference()

    n_vocab = len(set(durations))
    
    network_input, network_output = prepare_sequences(durations, n_vocab)
    
    model = create_network(network_input, n_vocab)
 
    checkpoint = ModelCheckpoint(
        "final-duration-weights.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    
    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(network_input, network_output, epochs=4, batch_size=3000, callbacks=callbacks_list)

train_duration_network()

Parsing COPY/dean-invisible.mid
Parsing COPY/IU-Love-Poem.mid
Parsing COPY/pick-me.mid
Parsing COPY/hangzoo-red-sun.mid
Parsing COPY/btob-ill-be-your-man.mid
Parsing COPY/eric-you-who.mid
Parsing COPY/IU-BBIBBI.mid
Parsing COPY/fx-sorry-dear-daddy.mid
Parsing COPY/vromance-im-fine.mid
Parsing COPY/infinite-bad.mid
Parsing COPY/nuest-w-just-one-day.mid
Parsing COPY/girls-day-ill-be-yours.mid
Parsing COPY/chief.-be-here-now.mid
Parsing COPY/half-moon.mid
Parsing COPY/BTS (방탄소년단) feat. Halsey - 작은 것들을 위한 시 (Boy With Luv)  (midi by Carlo Prato) (www.cprato.com).mid
Parsing COPY/zion-t-the-song.mid
Parsing COPY/blackpink.mid
Parsing COPY/Dreamcatcher-Deja-Vu.mid
Parsing COPY/block-b-yesterday.mid
Parsing COPY/btob-someday.mid
Parsing COPY/bigbang-last-dance.mid
Parsing COPY/dean-come-over.mid
Parsing COPY/iu-heart.mid
Parsing COPY/2A - San E - Bad Year.mid
Parsing COPY/sejeong-flower-road.mid
Parsing COPY/dean-what2do.mid
Parsing COPY/gfriend-fingertip.mid
Parsing COPY/bolbbagan4-tell-me-yo

KeyboardInterrupt: 

In [191]:
def train_Chorus_network():
    """ Train a Neural Network to generate music """
    Intro, Verse, Chorus, Outro = group_by_timestamp()

    n_vocab = len(set(Chorus))
    
    network_input, network_output = prepare_sequences(Chorus, n_vocab)
    
    model = create_network(network_input, n_vocab)
 
    checkpoint = ModelCheckpoint(
        "final-Chorus-weights.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    
    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(network_input, network_output, epochs=4, batch_size=50, callbacks=callbacks_list)

train_Chorus_network()

Parsing COPY/dean-invisible.mid
Parsing COPY/IU-Love-Poem.mid
Parsing COPY/pick-me.mid
Parsing COPY/hangzoo-red-sun.mid
Parsing COPY/btob-ill-be-your-man.mid
Parsing COPY/eric-you-who.mid
Parsing COPY/IU-BBIBBI.mid
Parsing COPY/fx-sorry-dear-daddy.mid
Parsing COPY/vromance-im-fine.mid
Parsing COPY/infinite-bad.mid
Parsing COPY/nuest-w-just-one-day.mid
Parsing COPY/girls-day-ill-be-yours.mid
Parsing COPY/chief.-be-here-now.mid
Parsing COPY/half-moon.mid
Parsing COPY/BTS (방탄소년단) feat. Halsey - 작은 것들을 위한 시 (Boy With Luv)  (midi by Carlo Prato) (www.cprato.com).mid
Parsing COPY/zion-t-the-song.mid
Parsing COPY/blackpink.mid
Parsing COPY/Dreamcatcher-Deja-Vu.mid
Parsing COPY/block-b-yesterday.mid
Parsing COPY/btob-someday.mid
Parsing COPY/bigbang-last-dance.mid
Parsing COPY/dean-come-over.mid
Parsing COPY/iu-heart.mid
Parsing COPY/2A - San E - Bad Year.mid
Parsing COPY/sejeong-flower-road.mid
Parsing COPY/dean-what2do.mid
Parsing COPY/gfriend-fingertip.mid
Parsing COPY/bolbbagan4-tell-me-yo

In [192]:
def train_Verse_network():
    """ Train a Neural Network to generate music """
    Intro, Verse, Chorus, Outro = group_by_timestamp()

    n_vocab = len(set(Verse))
    
    network_input, network_output = prepare_sequences(Verse, n_vocab)
    
    model = create_network(network_input, n_vocab)
 
    checkpoint = ModelCheckpoint(
        "final-Verse-weights.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    
    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(network_input, network_output, epochs=4, batch_size=50, callbacks=callbacks_list)

train_Verse_network()

Parsing COPY/dean-invisible.mid
Parsing COPY/IU-Love-Poem.mid
Parsing COPY/pick-me.mid
Parsing COPY/hangzoo-red-sun.mid
Parsing COPY/btob-ill-be-your-man.mid
Parsing COPY/eric-you-who.mid
Parsing COPY/IU-BBIBBI.mid
Parsing COPY/fx-sorry-dear-daddy.mid
Parsing COPY/vromance-im-fine.mid
Parsing COPY/infinite-bad.mid
Parsing COPY/nuest-w-just-one-day.mid
Parsing COPY/girls-day-ill-be-yours.mid
Parsing COPY/chief.-be-here-now.mid
Parsing COPY/half-moon.mid
Parsing COPY/BTS (방탄소년단) feat. Halsey - 작은 것들을 위한 시 (Boy With Luv)  (midi by Carlo Prato) (www.cprato.com).mid
Parsing COPY/zion-t-the-song.mid
Parsing COPY/blackpink.mid
Parsing COPY/Dreamcatcher-Deja-Vu.mid
Parsing COPY/block-b-yesterday.mid
Parsing COPY/btob-someday.mid
Parsing COPY/bigbang-last-dance.mid
Parsing COPY/dean-come-over.mid
Parsing COPY/iu-heart.mid
Parsing COPY/2A - San E - Bad Year.mid
Parsing COPY/sejeong-flower-road.mid
Parsing COPY/dean-what2do.mid
Parsing COPY/gfriend-fingertip.mid
Parsing COPY/bolbbagan4-tell-me-yo

In [193]:
def train_Intro_network():
    """ Train a Neural Network to generate music """
    Intro, Verse, Chorus, Outro = group_by_timestamp()

    n_vocab = len(set(Intro))
    
    network_input, network_output = prepare_sequences(Intro, n_vocab)
    
    model = create_network(network_input, n_vocab)
 
    checkpoint = ModelCheckpoint(
        "final-Intro-weights.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    
    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(network_input, network_output, epochs=4, batch_size=50, callbacks=callbacks_list)

train_Intro_network()

Parsing COPY/dean-invisible.mid
Parsing COPY/IU-Love-Poem.mid
Parsing COPY/pick-me.mid
Parsing COPY/hangzoo-red-sun.mid
Parsing COPY/btob-ill-be-your-man.mid
Parsing COPY/eric-you-who.mid
Parsing COPY/IU-BBIBBI.mid
Parsing COPY/fx-sorry-dear-daddy.mid
Parsing COPY/vromance-im-fine.mid
Parsing COPY/infinite-bad.mid
Parsing COPY/nuest-w-just-one-day.mid
Parsing COPY/girls-day-ill-be-yours.mid
Parsing COPY/chief.-be-here-now.mid
Parsing COPY/half-moon.mid
Parsing COPY/BTS (방탄소년단) feat. Halsey - 작은 것들을 위한 시 (Boy With Luv)  (midi by Carlo Prato) (www.cprato.com).mid
Parsing COPY/zion-t-the-song.mid
Parsing COPY/blackpink.mid
Parsing COPY/Dreamcatcher-Deja-Vu.mid
Parsing COPY/block-b-yesterday.mid
Parsing COPY/btob-someday.mid
Parsing COPY/bigbang-last-dance.mid
Parsing COPY/dean-come-over.mid
Parsing COPY/iu-heart.mid
Parsing COPY/2A - San E - Bad Year.mid
Parsing COPY/sejeong-flower-road.mid
Parsing COPY/dean-what2do.mid
Parsing COPY/gfriend-fingertip.mid
Parsing COPY/bolbbagan4-tell-me-yo

In [194]:
def train_Outro_network():
    """ Train a Neural Network to generate music """
    Intro, Verse, Chorus, Outro = group_by_timestamp()

    n_vocab = len(set(Outro))
    
    network_input, network_output = prepare_sequences(Outro, n_vocab)
    
    model = create_network(network_input, n_vocab)
 
    checkpoint = ModelCheckpoint(
        "final-Outro-weights.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    
    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(network_input, network_output, epochs=4, batch_size=50, callbacks=callbacks_list)

train_Outro_network()

Parsing COPY/dean-invisible.mid
Parsing COPY/IU-Love-Poem.mid
Parsing COPY/pick-me.mid
Parsing COPY/hangzoo-red-sun.mid
Parsing COPY/btob-ill-be-your-man.mid
Parsing COPY/eric-you-who.mid
Parsing COPY/IU-BBIBBI.mid
Parsing COPY/fx-sorry-dear-daddy.mid
Parsing COPY/vromance-im-fine.mid
Parsing COPY/infinite-bad.mid
Parsing COPY/nuest-w-just-one-day.mid
Parsing COPY/girls-day-ill-be-yours.mid
Parsing COPY/chief.-be-here-now.mid
Parsing COPY/half-moon.mid
Parsing COPY/BTS (방탄소년단) feat. Halsey - 작은 것들을 위한 시 (Boy With Luv)  (midi by Carlo Prato) (www.cprato.com).mid
Parsing COPY/zion-t-the-song.mid
Parsing COPY/blackpink.mid
Parsing COPY/Dreamcatcher-Deja-Vu.mid
Parsing COPY/block-b-yesterday.mid
Parsing COPY/btob-someday.mid
Parsing COPY/bigbang-last-dance.mid
Parsing COPY/dean-come-over.mid
Parsing COPY/iu-heart.mid
Parsing COPY/2A - San E - Bad Year.mid
Parsing COPY/sejeong-flower-road.mid
Parsing COPY/dean-what2do.mid
Parsing COPY/gfriend-fingertip.mid
Parsing COPY/bolbbagan4-tell-me-yo

In [195]:
def prepare_sequences_prediction(notes, pitchnames, n_vocab):

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    sequence_length = 40
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def choose_top(array):
    ind = np.argpartition(array, -8)[-8:]
    return (ind, array[ind])

def generate_notes(model, network_input, pitchnames, n_vocab, length):
    """ Generate notes from the neural network based on a sequence of notes """
    # Starts the melody by picking a random sequence from the input as a starting point
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    print(len(pattern))
    prediction_output = []

    for note_index in range(length):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        
        ### Copy the line below from your above implementation.
        prediction = model.predict(prediction_input)
        
        ind, array = choose_top(prediction[0])
        
        index = np.random.choice(ind)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]
            
    return prediction_output

In [196]:
def generateParts(filename, weights, length):
    notes = pickle.load(open(filename, 'rb'))
    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences_prediction(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    
    ### Add a line to load the weights here
    model.load_weights(weights)
    
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab, length)
    
    return prediction_output
    
def generate():
    prediction_output = []
    
    Intro = generateParts('Intro.p', 'final-Intro-weights.hdf5', 17)
    Verse = generateParts('Verse.p', 'final-Verse-weights.hdf5', 59)
    Chorus = generateParts('Chorus.p', 'final-Chorus-weights.hdf5',59)
    Outro = generateParts('Outro.p', 'final-Outro-weights.hdf5',35)
    
    prediction_output = Intro + Verse + Chorus + Verse + Chorus + Verse + Outro
    
    create_midi(prediction_output)

In [181]:
def generateDuration():
    durations = pickle.load(open('offset.p', 'rb'))
    pitchnames = sorted(set(item for item in durations))
    n_vocab = len(set(durations))

    network_input, normalized_input = prepare_sequences_prediction(durations, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    
    ### Add a line to load the weights here
    model.load_weights('final-duration-weights.hdf5')
    
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab, 347)
    
    return prediction_output

offset_seq = generateDuration()

40
347


In [224]:
def create_midi(prediction_output):
    offset = 0
    output_notes = []
    for index, pattern in enumerate(prediction_output):
        if ('.' in pattern) or pattern.isdigit():
            print("enter chord")
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            #new_chord.duration.quarterLength = test_duration[index]
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            #new_note.duration.quarterLength = test_duration[index]
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')
    
generate()

40
40
40
40
